## Two Tower Model 학습 시키기

### 1. Feature 테이블에서 데이터 읽어와서 train/test set으로 split 하기

In [6]:
import mysql.connector as sql

import pandas as pd

host = 'ssm-develop.db.sinsang.market'
port = 3306
username = 'dealicious'
password = 'tlstkd12!@'
database_name = 'dealicious'

db_connection = sql.connect(host=host, database=database_name, user=username, password=password)

retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)

retrievals_df.head(3)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_24648/1062528239.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)


,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided


In [7]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(retrievals_df, test_size=0.2, random_state=42)

In [7]:
train_df["article_id"] = train_df["article_id"].astype(str)  # to be removed
test_df["article_id"] = test_df["article_id"].astype(str)  # to be removed

pandas dataframe 을 tensorflow dataset 으로 변환하기

query embedding에 사용할 feature
- `customer_id`: ID of the customer.
- `age`: age of the customer at the time of purchase.
- `month_sin`, `month_cos`: time of year the purchase was made.

candidate embedding에 사용할 feature
- `article_id`: ID of the item.
- `garment_group_name`: type of garment.
- `index_group_name`: menswear/ladieswear etc.

In [8]:
import tensorflow as tf

query_features = ["customer_id", "age", "month_sin", "month_cos"]
candidate_features = ["article_id", "garment_group_name", "index_group_name"]

def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col : df[col] for col in df})

BATCH_SIZE = 448
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(test_df).batch(BATCH_SIZE).cache()

In [9]:
user_id_list = train_df["customer_id"].unique().tolist()
item_id_list = train_df["article_id"].unique().tolist()

garment_group_list = train_df["garment_group_name"].unique().tolist()
index_group_list = train_df["index_group_name"].unique().tolist()

print(f"Number of transactions: {len(train_df):,}")
print(f"Number of users: {len(user_id_list):,}")
print(f"Number of items: {len(item_id_list):,}")
print(garment_group_list)

Number of transactions: 14,784
Number of users: 3,845
Number of items: 6,487
['Jersey Fancy', 'Jersey Basic', 'Accessories', 'Knitwear', 'Trousers Denim', 'Socks and Tights', 'Special Offers', 'Under-, Nightwear', 'Blouses', 'Skirts', 'Trousers', 'Unknown', 'Swimwear', 'Shoes', 'Outdoor', 'Dresses/Skirts girls', 'Dressed', 'Dresses Ladies', 'Woven/Jersey/Knitted mix Baby', 'Shirts', 'Shorts']


### 2. Two Tower Model 학습

다음의 두 모델을 two tower 학습 기법을 통해 학습 시킴.

- `query model` : user, transaction feature를 나타내는 query embedding 생성하는 모델
- `candidate model` : item feature를 나타내는 candidate embedding을 생성하는 모델

In [10]:
EMB_DIM = 16

In [57]:
print(train_ds)

<_ShuffleDataset element_spec={'t_dat': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'customer_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'article_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'price': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'sales_channel_id': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'month_sin': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'month_cos': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'garment_group_name': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'index_group_name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>


In [11]:
from models.query_tower import QueryTower

query_model = QueryTower(user_id_list=user_id_list, emb_dim=EMB_DIM)
query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[query_features]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))


2024-05-02 14:39:01.745899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype double and shape [14784]
	 [[{{node Placeholder/_6}}]]
2024-05-02 14:39:01.746681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype double and shape [14784]
	 [[{{node Placeholder/_5}}]]
2024-05-02 14:39:02.040674: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [14

<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[ 0.09135035,  0.20549442, -0.4943238 ,  0.09084118,  0.2652784 ,
        -0.24944875, -0.11205667, -0.26309502, -0.19768474,  0.01405196,
        -0.16555972,  0.30976024, -0.0327182 , -0.23440479,  0.0403135 ,
         0.17018674]], dtype=float32)>

In [12]:
from models.item_tower import ItemTower


item_model = ItemTower(item_id_list=item_id_list, garment_group_list=garment_group_list, index_group_list=index_group_list, emb_dim=EMB_DIM)

item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)
item_ds = df_to_ds(item_df)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_24648/2937662747.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_df.drop_duplicates(subset="article_id", inplace=True)


In [13]:
import tensorflow_addons as tfa
from models.two_tower_model import TwoTowerModel

model = TwoTowerModel(query_model=query_model, item_model=item_model, item_ds=item_ds)
optimizer = tfa.optimizers.AdamW(0.001, learning_rate=0.01)
model.compile(optimizer=optimizer)

/Users/jimin/study/two-tower/.venv/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [14]:
model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5


2024-05-02 14:39:16.202678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype double and shape [14784]
	 [[{{node Placeholder/_7}}]]
2024-05-02 14:39:16.203314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int64 and shape [14784]
	 [[{{node Placeholder/_8}}]]


30/33 [==========================>...] - ETA: 0s - loss: 2690.0121 - regularization_loss: 0.0000e+00 - total_loss: 2690.0121

2024-05-02 14:39:17.941503: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_8' with dtype int64 and shape [3697]
	 [[{{node Placeholder/_8}}]]


33/33 [==============================] - 2s 22ms/step - loss: 2678.3572 - regularization_loss: 0.0000e+00 - total_loss: 2678.3572 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 499.9956 - val_regularization_loss: 0.0000e+00 - val_total_loss: 499.9956
Epoch 2/5
33/33 [==============================] - 0s 14ms/step - loss: 2413.8755 - regularization_loss: 0.0000e+00 - total_loss: 2413.8755 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 490.

In [17]:
print(model)

### 3. 모델 저장하기

In [15]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {"customer_id": instances["customer_id"],
                "month_sin": instances["month_sin"],
                "month_cos": instances["month_cos"],
                "query_emb": query_emb}

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [21]:
instances_spec={
    'customer_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='customer_id'),
    'month_sin': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_sin'),
    'month_cos': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_cos'),
    'age': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='age')
}
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

tf.saved_model.save(query_model, "query_model", signatures=signatures)

INFO:tensorflow:Assets written to: query_model/assets


In [22]:
tf.saved_model.save(model.item_model, "candidate_model")

INFO:tensorflow:Assets written to: candidate_model/assets


In [51]:
# 모델 로드
loaded_model = tf.saved_model.load("query_model")

# 예측 함수 추출
infer = loaded_model.signatures["serving_default"]

# 입력 스펙 확인
# print(infer.structured_input_signature)

# 샘플 입력 데이터 생성 (double 타입으로)
sample_input = {
    'customer_id': tf.constant(['customer1', 'customer2']),
    'month_sin': tf.constant([0.1, 0.2], dtype=tf.float64),
    'month_cos': tf.constant([0.2, 0.3], dtype=tf.float64),
    'age': tf.constant([30.0, 40.0], dtype=tf.float64)
}

# 예측 수행
predictions = infer(**sample_input)

# 예측 결과 출력
print(predictions)

{'month_sin': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.1, 0.2])>, 'query_emb': <tf.Tensor: shape=(2, 16), dtype=float32, numpy=
array([[-0.01002674,  0.1559282 ,  0.20848864, -0.29459476, -0.05245731,
         0.02505475, -0.40152204, -0.5368727 ,  0.10836559,  0.29850647,
        -0.55662155,  0.15531695, -0.18558472,  0.05173267,  0.27168903,
        -0.3877893 ],
       [-0.1436327 ,  0.19334501,  0.18951917, -0.31433272, -0.2350296 ,
         0.14532247, -0.26973787, -0.20852515,  0.2952414 , -0.19411935,
        -0.13977264,  0.14159653, -0.22866179,  0.02863191, -0.23835948,
        -0.06198785]], dtype=float32)>, 'customer_id': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'customer1', b'customer2'], dtype=object)>, 'month_cos': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.2, 0.3])>}


In [34]:
import boto3

AWS_PROFILE = 'default'
session = boto3.Session(profile_name=AWS_PROFILE)
s3_client = session.client('s3')

{'ResponseMetadata': {'RequestId': '5TKGWRAEY1GESDNF', 'HostId': 'UeNCHD4Qm9w88G7SPkA/fb5z0rjua4AcOHxIVOTzLKAhRK0X0+BDsIQ2Ldm9Gd9bGLiHo/Tmf8w=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'UeNCHD4Qm9w88G7SPkA/fb5z0rjua4AcOHxIVOTzLKAhRK0X0+BDsIQ2Ldm9Gd9bGLiHo/Tmf8w=', 'x-amz-request-id': '5TKGWRAEY1GESDNF', 'date': 'Thu, 02 May 2024 06:14:01 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'jimin-model', 'CreationDate': datetime.datetime(2024, 4, 22, 10, 46, 4, tzinfo=tzutc())}], 'Owner': {'ID': '3da57189eb51f3b76f1ecb9eb6413d9dfc082c729fcb356bb25a368d6921add5'}}


In [ ]:
local_model_path = 'candidate_model'
bucket = 'jimin_model'
key = 'two-tower'
s3_client.upload_file(local_model_path, bucket, key)

In [54]:
import os
import tarfile
import boto3

def make_tar_gz(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

def upload_to_s3(local_file_path, bucket_name, s3_file_path):
    s3_client.upload_file(local_file_path, bucket_name, s3_file_path)

def zip_and_upload_to_s3(local_directory, bucket_name, s3_directory):
    # 로컬 디렉토리를 tar.gz로 압축
    tar_gz_filename = local_directory + ".tar.gz"
    make_tar_gz(tar_gz_filename, local_directory)

    # S3에 업로드할 경로 설정
    s3_file_path = os.path.join(s3_directory, os.path.basename(tar_gz_filename))

    # 압축 파일을 S3로 업로드
    upload_to_s3(tar_gz_filename, bucket_name, s3_file_path)

    # 압축 파일 삭제
    os.remove(tar_gz_filename)
    print(f"Uploaded {tar_gz_filename} to s3://{bucket_name}/{s3_file_path}")

# 로컬 디렉토리 경로
local_directory = "query_model"

# S3 버킷 이름과 업로드할 경로
bucket_name = "jimin-model"
s3_directory = "two-tower"

# 디렉토리를 tar.gz로 압축하고 S3로 업로드
zip_and_upload_to_s3(local_directory, bucket_name, s3_directory)


Uploaded query_model.tar.gz to s3://jimin-model/two-tower/query_model.tar.gz


In [60]:
s3_client.download_file('jimin-model', 'two-tower/query_model.tar.gz', 'temp/query_model.tar.gz')

In [66]:
import os
extract_dir = os.path.splitext('temp/query_model.tar.gz')[0]
print(extract_dir)

temp/query_model.tar


In [68]:
# importing the "tarfile" module 
import tarfile 
  
# open file 
file = tarfile.open('temp/query_model.tar.gz') 
  
# print file names 
print(file.getnames()) 
  
# extract files 
file.extractall('./temp') 
  
# close file 
file.close() 

['query_model', 'query_model/assets', 'query_model/fingerprint.pb', 'query_model/saved_model.pb', 'query_model/variables', 'query_model/variables/variables.data-00000-of-00001', 'query_model/variables/variables.index']


In [67]:
import os
import tarfile
import boto3
import tensorflow as tf

def download_from_s3(bucket_name, s3_file_path, local_directory):
    local_file_path = os.path.join(local_directory, os.path.basename(s3_file_path))
    s3_client.download_file(bucket_name, s3_file_path, local_directory)
    return local_file_path

def extract_tar_gz(tar_gz_file, extract_dir):
    with tarfile.open(tar_gz_file, "r:gz") as tar:
        tar.extractall(path=extract_dir)

def load_model_from_s3(bucket_name, s3_file_path, local_directory):
    # S3에서 tar.gz 파일 다운로드
    # local_tar_gz_file = download_from_s3(bucket_name, s3_file_path, local_directory)
    # print(local_tar_gz_file)
    local_tar_gz_file = 'temp/query_model.tar.gz'

    # tar.gz 파일 압축 해제
    extract_dir = os.path.splitext(local_tar_gz_file)[0]
    extract_tar_gz(local_tar_gz_file, extract_dir)

    # 압축 해제된 모델 로드
    loaded_model = tf.saved_model.load(extract_dir)

    return loaded_model

# S3 버킷 이름과 파일 경로
bucket_name = "jimin-model"
s3_file_path = "two-tower/query_model.tar.gz"

# 로컬에 저장될 디렉토리 경로
local_directory = "temp"

# 모델 로드
loaded_model = load_model_from_s3(bucket_name, s3_file_path, local_directory)

# 예측 함수 추출
infer = loaded_model.signatures["serving_default"]

# 입력 스펙 확인
# print(infer.structured_input_signature)

# 샘플 입력 데이터 생성 (double 타입으로)
sample_input = {
    'customer_id': tf.constant(['customer1', 'customer2']),
    'month_sin': tf.constant([0.1, 0.2], dtype=tf.float64),
    'month_cos': tf.constant([0.2, 0.3], dtype=tf.float64),
    'age': tf.constant([30.0, 40.0], dtype=tf.float64)
}

# 예측 수행
predictions = infer(**sample_input)

# 예측 결과 출력
print(predictions)


OSError: SavedModel file does not exist at: temp/query_model.tar/{saved_model.pbtxt|saved_model.pb}